# Imports

In [139]:
%pip install matplotlib
%pip install scikit-learn
%pip install scipy
%pip install tensorflow
%pip install numpy
%pip install pandas
%pip install os
%pip install setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [140]:
import tensorflow as tf

from tensorflow.image import resize
from tensorflow.keras.backend import clear_session
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.metrics import  Recall, CategoricalAccuracy
from IPython.display import clear_output

from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import concatenate as concat
from scipy.stats import entropy
import os

from helpers.help import *


np.random.seed(0)


# Start

In [141]:
# Open Diabetic Retinopathy dataset
path = os.path.join(os.getcwd(),'gaussian_ds')
label_dict={'Mild':1,'Moderate':1,'Proliferate_DR':1,'Severe':1,'No_DR':0}

# remove macOS file
folders = os.listdir(path)
folders.remove('.DS_Store')

# get all the samples
array = []
for i in folders:
    detailPath = os.path.join(path,i)
    for j in os.listdir(detailPath):
        array.append([os.path.join(detailPath,j),label_dict[i.split('.')[0]]])

# transforms the array into nparray
dataset=np.array(array)

np.size(dataset,0)

dataset[0:3]

array([['/Users/leonardosousa/Desktop/mestrado/IA/Project/RetinopathyAlgorithm/gaussian_ds/Mild/2d7666b8884f.png',
        '1'],
       ['/Users/leonardosousa/Desktop/mestrado/IA/Project/RetinopathyAlgorithm/gaussian_ds/Mild/50840c36f0b4.png',
        '1'],
       ['/Users/leonardosousa/Desktop/mestrado/IA/Project/RetinopathyAlgorithm/gaussian_ds/Mild/30cab14951ac.png',
        '1']], dtype='<U113')

# Pre-Processing

In [142]:
# Get characteristics and labels (ensured they have the right type)
X,y=dataset[:,0],dataset[:,1]
y = y.astype(int)
y = to_categorical(y)

#Shuffle the dataset (to make a unbiased model)
p = np.random.permutation(len(X))
X,y = X[p], y[p]

#Strip off 10% samples for hold out test set
test_idxs = np.random.choice(len(X), size=int(0.1*len(X)), replace=False, p=None)
x_test, y_test = X[test_idxs],y[test_idxs]
#Delete the test set samples from X,y 
X = np.delete(X, test_idxs)
y = np.delete(y, test_idxs, axis = 0)

#usual train-val split. We use 11% here just match the test set size to validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.11)

abc

# Separate the Seed and Pools
- **Seeds**: what is used to start the training
- **Pool**: serve as a pool for uncertainty sampling

In [143]:
initial_seed = 100
x_seed , x_pool = x_train[0:initial_seed], x_train[initial_seed:]
y_seed , y_pool = y_train[0:initial_seed], y_train[initial_seed:]

print(f"Samples in Seed set: {x_seed.shape[0]}")
print(f"Samples in Pool: {x_pool.shape[0]}")
print(f"Samples in Validation set: {x_val.shape[0]}")
print(f"Samples in Test set: {x_test.shape[0]}")

Samples in Seed set: 100
Samples in Pool: 2833
Samples in Validation set: 363
Samples in Test set: 366


abc

In [144]:
#The buid_dataset is a custom function that returns tensor batches

val_dataset=build_dataset(x_val,y_val,repeat=False,batch=256)
test_dataset=build_dataset(x_test,y_test,repeat=False,batch=256)
pool_dataset=build_dataset(x_pool,y_pool,repeat=False,batch=256, shuffle = False)

BATCH_SIZE=16
STEPS_PER_EPOCH=len(x_train)/BATCH_SIZE

train_dataset=build_dataset(x_seed,y_seed,batch=BATCH_SIZE)
input_shape=train_dataset.element_spec[0].shape[1:]

Build seed model

In [145]:
print(input_shape)
model=simple_model(input_shape)
model.compile(
        loss = "categorical_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )
model.summary()


(64, 64, 3)


/Users/leonardosousa/Desktop/mestrado/IA/Project/RetinopathyAlgorithm/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 13, 13, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 684,866 (2.61 MB)

 Trainable params: 684,162 (2.61 MB)

 Non-trainable params: 704 (2.75 KB)

In [146]:
# Saves the best model
checkpoint=ModelCheckpoint(filepath='model/model_al.keras',
                           monitor='val_loss',save_best_only=True,verbose=1)

# Logs the progress of training
csv_logger=keras.callbacks.CSVLogger('logger/trainlog_al.csv',
                                     separator=',',append=False)

# prevent overfitting
early_stopper=keras.callbacks.EarlyStopping(monitor='val_loss',
                                            min_delta=0.001,
                                            restore_best_weights=True,
                                            patience=10)

callbacks_list=[checkpoint,early_stopper,csv_logger]

# Training

In [147]:
EPOCHS = 20
model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=EPOCHS,
          validation_data=val_dataset,validation_steps=None,
          callbacks=callbacks_list)

Epoch 1/20
182/183 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 0.9261 - loss: 0.2265
Epoch 1: val_loss improved from inf to 5.65988, saving model to model/model_al.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - categorical_accuracy: 0.9264 - loss: 0.2255 - val_categorical_accuracy: 0.4904 - val_loss: 5.6599
Epoch 2/20
182/183 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - categorical_accuracy: 0.9883 - loss: 0.0326
Epoch 2: val_loss improved from 5.65988 to 2.89406, saving model to model/model_al.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - categorical_accuracy: 0.9883 - loss: 0.0325 - val_categorical_accuracy: 0.4986 - val_loss: 2.8941
Epoch 3/20
182/183 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 0.9919 - loss: 0.0221
Epoch 3: val_loss did not improve from 2.89406
183/183 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - categorical_accuracy: 0.9919 - loss: 0.0220 - val_categorical_accuracy: 0.5014 - val_loss: 4.9176
Epoch 4/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - c

# Initial Model Evaluation on Test Dataset (Smaller Dataset / Seeded)

In [148]:
model = keras.models.load_model("model/model_al.keras")
print("-" * 100)
print(model.evaluate(test_dataset, verbose=0,return_dict=True))

----------------------------------------------------------------------------------------------------
{'categorical_accuracy': 0.9098360538482666, 'loss': 0.469754159450531}


# Entering AL Loop
We will now iteratively query the pool for samples and add them to the seed set. Every time we can pick sampling_size number of points from the pool with the largest entropy.

Step_1: test the current model on the test set. If it exceeds or equals the baseline accuracy , then we exit the AL loop. Otherwise we proceed.
Step_2: measure the uncertainties in the pool dataset. In other words we query the pool dataset. For this experiment, we will use the entropy measure. Pick the top 200 samples with maximum entropy, append them to the seed dataset and delete them from the pool.
Step_3: re-compile the model to reset the optimizer states and fit again. Save model if there is an improvement in loss. Go back to Step_1


In [149]:
sampling_size=100
num_iterations = int(x_pool.shape[0]/sampling_size)
print(x_pool.shape[0])
print(sampling_size)
al_history = []
csv_logger=keras.callbacks.CSVLogger('logger/trainlog_al.csv',
                                 separator=',',append=True)
callbacks_list=[checkpoint,early_stopper,csv_logger]


2833
100


# Include baseline (full train) model

In [150]:
try:
    
    model_full = keras.models.load_model("model/model_baseline.keras")
    _, acc_baseline = model_full.evaluate(test_dataset)

    print("-" * 100)
    print(model_full.evaluate(test_dataset, verbose=0,return_dict=True))
    
except FileNotFoundError:
    print("model file model_baseline.h5 not found. Make sure to run 01_Training_Full.ipynb entirely")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - categorical_accuracy: 0.9180 - loss: 0.1981
----------------------------------------------------------------------------------------------------
{'categorical_accuracy': 0.9180327653884888, 'loss': 0.19902613759040833}


In [152]:
acc_baseline = round(acc_baseline,4)

#reducing steps:
num_iterations = int(num_iterations -20)
print(num_iterations)
for iteration in range(num_iterations):
    
    #Step_1
    loss, acc = model.evaluate(test_dataset, verbose=0)
    print(f"Test Set Accuracy after {iteration} iteration {acc}")
    al_history.append([loss, acc, x_seed.shape[0], x_pool.shape[0]])
    
    if acc_baseline - 0.0025 < acc <= acc_baseline + 0.0025:
        print("Terminating Training")
        break
    
    #Step_2
    #Use the current model to predict the pool dataset
    print("Predicting pool dataset")
    y_pool_proba = model.predict(pool_dataset)
    
    #Pick the index of the top entropy samples in pool
    pool_max_ents = np.argsort(entropy(y_pool_proba.T))[-sampling_size:]
    
    #Acquire those samples from pool
    x_sample = x_pool[pool_max_ents]
    y_sample = y_pool[pool_max_ents]
    
    #Add these samples to the seed dataset
    y_seed = concat((y_seed,y_sample),axis=0)
    x_seed = concat((x_seed,x_sample),axis=0)
     
    #Delete the acquired samples from pool
    x_pool = np.delete(x_pool, pool_max_ents, 0 )
    y_pool = np.delete(y_pool, pool_max_ents, 0 )

    #Build the tensorflow dataset object for this iteration
    pool_dataset = build_dataset(x_pool,y_pool,repeat=False,batch=256,
                                 shuffle = False)
    train_dataset = build_dataset(x_seed,y_seed,batch=BATCH_SIZE) 

    print(f"Samples in seed dataset {x_seed.shape[0]} , in pool dataset {x_pool.shape[0]}")
    print("-" * 100)

    #Step_3
    model.compile(
        loss = "binary_crossentropy",
        optimizer = Adam(),
        metrics=[CategoricalAccuracy()]
    )
    
    history = model.fit(train_dataset,steps_per_epoch=int(STEPS_PER_EPOCH),epochs=10,
          validation_data=val_dataset,validation_steps=None,
          callbacks=callbacks_list)
    
    #If the fit method generated a new best model , load it for
    #the next iteration
    model = keras.models.load_model("model/model_al.keras")
    clear_output()
    clear_session()

Test Set Accuracy after 3 iteration 0.9180327653884888
Terminating Training


In [153]:
df = pd.DataFrame(al_history, columns = ['Test Loss', 'Test Accuracy', 'Seed', 'Pool'])
df

,Test Loss,Test Accuracy,Seed,Pool
0,0.469754,0.909836,100,2833
1,4.199880,0.543716,200,2733
2,0.204831,0.942623,300,2633
3,0.204831,0.942623,400,2533
4,0.261787,0.918033,500,2433


In [154]:
df.to_csv('logger/AL_tracking.csv', index = False)